# COVID-19 Forecasting

In [ ]:
#Mount drive on colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#change directory to project folder
%cd drive/MyDrive/COVID-19

/content/drive/MyDrive/COVID-19


## Import Libraries

In [ ]:
import os
import platform
from datetime import datetime
import time
import subprocess
import pandas as pd
import hashlib
import numpy as np
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Reduce tensorflow messages.
import logging
from sklearn.metrics import mean_squared_error, mean_absolute_error
import csv
from tabulate import tabulate
import matplotlib.pyplot as plt

## Data Loader

In [ ]:
class cd:
    # class to change current working directory for colab
    def __init__(self, newPath):
        self.newPath = os.path.expanduser(newPath)

    def __enter__(self):
        self.savedPath = os.getcwd()
        os.chdir(self.newPath)

    def __exit__(self, etype, value, traceback):
        os.chdir(self.savedPath)

In [ ]:
class DataLoader:
    #class to load data
    def __init__(self):
        self.covid_dir = "COVID-19/csse_covid_19_data/csse_covid_19_time_series/" #project directory
        # Don't change the order of the items in this list.
        self.files = ["time_series_covid19_confirmed_global.csv", "time_series_covid19_deaths_global.csv",
                      "time_series_covid19_recovered_global.csv"] #files for the three timeseries
        

    def load_population_data(self):
        #load population data from the kaggle dataset
        pop = pd.read_csv("kaggle_data/covid19-global-forecasting-week-5/train.csv")
        # Remove unnecessary columns.
        pop.drop(columns=["Id", "County", "Province_State", "Weight", "Date", "Target", "TargetValue"], inplace=True)
        # Find country's population
        return pop.groupby(['Country_Region']).max()

    def load_covid_data(self):
        # load data from all the csv files
        # download data from github repository if path doesn't exist
        if not os.path.isdir(self.covid_dir):
            subprocess.call(["git", "clone", "https://github.com/CSSEGISandData/COVID-19.git"])

        # Create a list of paths from the file names.
        paths = [os.path.join(self.covid_dir, f) for f in self.files]

        # Get new data
        if platform.system() == 'Linux':  # TODO make cross-platform.
            stats = [os.stat(path) for path in paths]
            # Get today's date.
            today = datetime.utcfromtimestamp(int(time.time())).strftime('%Y-%m-%d')
            # Get last modified date for each file.
            dates = [datetime.utcfromtimestamp(int(stat.st_mtime)).strftime('%Y-%m-%d') for stat in stats]
            # Check whether all data files have been modified today. If not get updates.
            if not all(today == creation_time for creation_time in dates):
                with cd("COVID-19"):
                    subprocess.call(["git", "pull", "origin", "master"])
        else:
            print(f"Platform not supported: {platform.system()}")
            raise SystemExit

        # Load all csv files into a list of data frames.
        data_frames = [pd.read_csv(path) for path in paths]

        # Remove unnecessary columns and return dataframes.
        return [df.drop(columns=['Province/State', 'Lat', 'Long']) for df in data_frames]  # Confirmed, Dead and Recovered.

## Country

In [ ]:
class Country:
    """ Represents a country with COVID-19 """

    def __init__(self, n, p, c, d, r):
        """
            Params:
                n - string - The country's name.
                p - int - The country's population.
                c - pandas Series - Time series of the confirmed cases.
                d - pandas Series - Time series of the deseased cases.
                r - pandas Series - Time series of the recovered cases.
        """
        self.name = n
        self.encoded_name = None
        self.population = p
        self.data = pd.concat([c, d, r], axis=1)
        self.data.columns=["Confirmed", "Deceased", "Recovered"]

    def find_divisor_offset(self, div):
        """
        Calculates an offset to make the data divisible by the horizon.
        By finding the greatest divisor of div that is smaller than num.

        div  - int - horizon
        returns offset - int

        Example: If splitting the data into weeks.
        num = 137
        div = 7
        137/7 = 19.57... (Not divisible)
        Using the formula below:
        137 - (floor(137/7) * 7) = 4
        Four is the offset to make 137 divisible by 7 because
        137 - 4 = 133
        133/7 = 19 (Divisible)
        """
        # Find how many days have to be cut of the start to make the data divisible by the horizon.
        # This is required as the cross validation folds must the the same size.
        time_steps = len(self.data)
        return time_steps - ((time_steps//div)*div)

    def split_k_fold(self, train_size, horizon):
        """
        train_size - int - number of time steps in the train data.
        horizon - int - number of time steps to be forecasted.
        """

        # Make the data divisible by the horizon to guarantee all k folds have the same size.
        # Cuts out the old data instead of the new time steps.
        offset = self.find_divisor_offset(horizon)
        temp_data = self.data[offset:]

        val_end = train_size + horizon
        test_end = train_size + horizon * 2

        train = temp_data[:train_size]
        val = temp_data[train_size:val_end]
        test_x = temp_data[:val_end]
        test_y = temp_data[val_end:test_end]

        return train, val, test_x, test_y

## Data

In [ ]:
class Data:
    """ A container that stores the time series COVID-19 data for each country.
        The data is loaded from the data_loader module.
    """
    def __init__(self):
        self.loader = DataLoader()
        self.raw_confirmed, self.raw_deceased, self.raw_recovered = self.loader.load_covid_data()
        self.population = self.loader.load_population_data()
        self.countries = []
        self.save_dir = "cross_val_results"
        self.pad_val = -10000
        self.horizon = 28
        self.features = 3
        self.padding = np.full((self.horizon, self.features), self.pad_val)

    def prepare_data(self):
        self.populate_countries()
        self.encode_names()
        self.split_data()
        self.save_data()

        scaled_train, _ = self.standardize(self.train)
        scaled_val, self.val_scalers = self.standardize(self.val)
        scaled_test_x, _ = self.standardize(self.test_x)
        scaled_test_y, self.test_y_scalers = self.standardize(self.test_y)

        self.padded_scaled_train = self.pad_data(scaled_train)
        self.padded_scaled_test_x = self.pad_data(scaled_test_x, offset=1)

        self.multi_out_scaled_val = self.prepare_output_data(scaled_val)
        self.multi_out_scaled_test_y = self.prepare_output_data(scaled_test_y)

    def populate_countries(self):
        """
            Creates a list of Country objects each containing the COVID-19 data for their respective country.
            Retunrs:
                A list of Country objects.
        """
        # For each country in population.
        for name, pop in self.population.iterrows():
            p = pop['Population']
            # Get all relevant time series based on country name.
            c = self.raw_confirmed.loc[self.raw_confirmed['Country/Region'] == name].sum(numeric_only=True)
            d = self.raw_deceased.loc[self.raw_deceased['Country/Region'] == name].sum(numeric_only=True)
            r = self.raw_recovered.loc[self.raw_recovered['Country/Region'] == name].sum(numeric_only=True)
            # Create new country object.
            self.countries.append(Country(name, p, c, d, r))

    def encode_names(self, output_space=6):
        """
        Creates hashed versions of the country names.
        These will be used in the Embedding layer of the network.
        Param: output_space - int - size of the poutput produced by the hash functions to guarantee uniqueness.
        Returns:
        hashed_names - list - a list containing all the hashed names for the countries.
        """
        # Hash all country names using SHA-256 then convert the hex output to int slice of the first digits by converting it to
        # a string and save it as an int in a list.
        hashed_names = [int(str(int(hashlib.sha256(country.encode('utf-8')).hexdigest(), 16))[:output_space]) for country in
                        self.population.index]

        # Convert the integers into an array of digits.
        hashed_names = np.stack([np.array(list(map(int,str(x)))) for x in hashed_names])

        for country, hashed_name in zip(self.countries, hashed_names):
            country.encoded_name = hashed_name

        self.encoded_names = hashed_names

    def cross_validate(self, train_size):
        """
        Cross validate data for all countries.
        """
        train, val, test_x, test_y = [], [], [], []
        for country in self.countries:
            tr, v, te_x, te_y = country.split_k_fold(train_size, self.horizon)
            train.append(tr), val.append(v), test_x.append(te_x), test_y.append(te_y)
        return np.stack(train), np.stack(val), np.stack(test_x), np.stack(test_y)

    def split_data(self):
        """
        Returns four lists of numpy arrrays. Each list has the shape (fold, countries, days). The number of days changes over the
        folds.
        """
        self.train, self.val, self.test_x, self.test_y = [], [], [], []
        train_size = self.horizon
        # This assumes all countries have the same length.
        # The minus two gives space for the validation and test sets as they will overshoot.
        k_folds = len(self.countries[0].data)//self.horizon - 2
        for _ in range(k_folds):
            tr, v, te_x, te_y = self.cross_validate(train_size)
            self.train.append(tr), self.val.append(v), self.test_x.append(te_x), self.test_y.append(te_y)
            train_size += self.horizon

    def save_to_npz_folds(self, data, file_name):
        """
        Parameters:
            data - list containing numpy arrays with length = folds.
            file_name - string.
        """
        file_path = os.path.join(self.save_dir, file_name)
        with open(file_path, "w+b") as new_file:
            np.savez(new_file, *data)

    def save_data(self):
        #save data to npz file
        self.save_to_npz_folds(self.train, "train")
        self.save_to_npz_folds(self.val, "val")
        self.save_to_npz_folds(self.test_x, "test_x")
        self.save_to_npz_folds(self.test_y, "test_y")

    def standardize(self, data):
        #scale data using Standard Scaler
        scaled_data = []
        scalers = []
        for fold in data:
            scaled_fold = []
            scalers_fold = []
            for country in fold:
                scaler = StandardScaler()
                scaled_fold.append(scaler.fit_transform(country))
                scalers_fold.append(scaler)
            scaled_data.append(np.stack(scaled_fold))
            scalers.append(scalers_fold)
        return scaled_data, scalers

    def pad_data(self, data, offset=0):
        #pad data
        padded_scaled_data = []
        folds = len(data) - offset
        for fold in data:
            padded_fold = []
            fold_padding = np.repeat(self.padding, folds).reshape(self.horizon*folds, self.features)
            for row in fold:
                padded_fold.append(np.append(row, fold_padding, axis=0))
            folds -= 1
            padded_scaled_data.append(np.stack(padded_fold))
        return padded_scaled_data

    def make_multi_output_data(self, data):
        """
        Takes a dataset with shape (sample, timestep, feature) and
        reshapes it to (feature, sample, timestep)
        Returns: 3D numpy array
        """
        confirmed, deceased, recovered = [], [], []
        for sample in data:
            confirmed.append(sample[:,0])
            deceased.append(sample[:,1])
            recovered.append(sample[:,2])
        confirmed = np.stack(confirmed)
        deceased = np.stack(deceased)
        recovered = np.stack(recovered)
        return np.stack([confirmed, deceased, recovered])

    def prepare_output_data(self, data):
        multi_out_data = []
        for fold in data:
            multi_out_data.append(self.make_multi_output_data(fold))
        return multi_out_data

## Model

In [ ]:
class EncoderBlock(layers.Layer):
    """
    Encoder block that takes as input a time series and a numerial representation of a county name
    and creates a learned representation to be processed further in the model.
    """
    def __init__(self, rnn_units, rnn_layer, rnn_activation, pad_val, l1=0, l2=0, dropout=0):
        super().__init__()
        regularizer = tf.keras.regularizers.L1L2(l1=l1, l2=l2)

        self.mask_layer = None
        if pad_val:
            self.mask_layer = layers.Masking(mask_value=pad_val)
        #defined encoder architecture
        self.hidden_rnn = rnn_layer(rnn_units, activation=rnn_activation, kernel_regularizer=regularizer, dropout=dropout,
                                    name="rnn_encoder")
        self.hidden_dense = layers.Dense(1, name="name_encoder")

    def call(self, inputs):
        if self.mask_layer:
            masked_inputs = self.mask_layer(inputs[0])
        #call the architecture
        h_rnn = self.hidden_rnn(masked_inputs)
        h_dense = self.hidden_dense(inputs[1])

        return layers.concatenate([h_rnn, h_dense], name="context")

In [ ]:
class MultiOutputRNN(keras.Model):
    """
    Multi output RNN model with individual weights on the output nodes.
    """
    def __init__(self, output_size, rnn_units, rnn_layer, rnn_activation, l1, l2, dropout, pad_val=None):
        super().__init__()
        self.encoder = EncoderBlock(rnn_units, rnn_layer, rnn_activation, pad_val, l1, l2, dropout)
        #decoder block
        self.c_out = layers.Dense(output_size, name="confirmed")
        self.d_out = layers.Dense(output_size, name="deceased")
        self.r_out = layers.Dense(output_size, name="recovered")

    def call(self, inputs):
        context = self.encoder(inputs)
        return self.c_out(context), self.d_out(context), self.r_out(context)

In [ ]:
class Model():
    def __init__(self, output_size, rnn_units, rnn_layer, rnn_activation, l1, l2, dropout, pad_val):
        #set model according to parameters passed
        self.model = MultiOutputRNN(output_size, rnn_units, rnn_layer, rnn_activation, l1, l2, dropout, pad_val)
    #compile model
    def compile(self, optimizer, loss, metrics):
        self.model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    #fit model
    def fit(self, x, y, epochs, callbacks=None, verbose=2):
        return self.model.fit(x=x, y=y, epochs=epochs, callbacks=callbacks, verbose=verbose)
    #evaluate model
    def evaluate(self, x, y, verbose=1, return_dict=True):
        return self.model.evaluate(x=x, y=y, verbose=verbose, return_dict=return_dict)
    #test model
    def predict(self, inputs):
        return self.model.predict(inputs)

## Experiment

In [ ]:
class Experiment():
    #main class where all the experiments are performed
    def __init__(self, val_scalers, test_scalers):
        self.logger = logging.getLogger("experiment") #log process
        self.save_dir = "cross_val_results" #save directory
        self.ensemble_size = 1
        self.units = 20 #number of units
        self.epochs = 300 #number of epochs
        self.val_scalers = val_scalers 
        self.test_scalers = test_scalers
        # layers and functions defines
        self.tanh = tf.keras.activations.tanh
        self.lstm = tf.keras.layers.LSTM
        self.gru = tf.keras.layers.GRU
        self.adam = tf.keras.optimizers.legacy.Adam()
        self.mse_loss = tf.keras.losses.MeanSquaredError()
        self.metrics = [tf.keras.metrics.MeanSquaredError(), tf.keras.metrics.RootMeanSquaredError()]
        # Regularization methods according to table in research paper
        self.regularizers = [[0,    0,    0],    # No regularization
                             [0.01, 0,    0],    # Default L1 only.
                             [0,    0.01, 0],    # Default L2 only.
                             [0,    0,    0.2],  # Small dropout only.
                             [0.01, 0.01, 0],    # Default L1 and L2, no dropout.
                             [0.01, 0.01, 0.2]   # All regularizers.
                            ]

    def destandardize_data(self, data, scalers):
        """
        data - numpy array of shape (countries, horizon, features)
        returns numpy array of shape shape as data
        """
        rescaled_data = []
        for country_pred, scaler in zip(data, scalers):
            rescaled_data.append(scaler.inverse_transform(country_pred))
        return np.stack(rescaled_data)

    def prepare_predictions(self, data, test_scalers):
        """
        data - list - contains one numpy array of shape (countries, horizon) for each feature.
        returns numpy array of shape (countries, horizon, features) with rescaled data.
        """
        # First reshape the data.
        reshaped_preds = []
        # Unpack all sub lists into the three variables.
        for c, d, r in zip(*data):
            reshaped_preds.append(np.stack([c, d, r]).T)
        reshaped_preds = np.stack(reshaped_preds)

        return self.destandardize_data(reshaped_preds, test_scalers)

    def calculate_rmse(self, orig, pred):
        """
        orig - numpy array of shape (countries, horizon, features)
        pred - numpy array of shape (countries, horizon, features)
        Returns numpy array of shape (countries, features) containing the RMSE of all predictions
        """
        results = []
        for o, p in zip(orig, pred):
            results.append(mean_squared_error(o, p, multioutput='raw_values', squared=False))
        return np.stack(results)

    def calculate_mae(self, orig, pred):
        """
        orig - numpy array of shape (countries, horizon, features)
        pred - numpy array of shape (countries, horizon, features)
        Returns numpy array of shape (countries, features) containing the MAE of all predictions
        """
        results = []
        for o, p in zip(orig, pred):
            results.append(mean_absolute_error(o, p, multioutput='raw_values'))
        return np.stack(results)

    def save_to_npz(self, data, file_name, exp_name):
        #save npz file
        file_path = os.path.join(self.save_dir, f"{file_name}_{exp_name}")
        with open(file_path, "w+b") as new_file:
            np.savez(new_file, data)

    def run_experiments(self, horizon, padding, train, val, test_x, test_y, orig_val, orig_test_y, enc_names):
        #run experiments mentioned in paper
        fold_idx = 0
        data_zip = zip(train, val, test_x, test_y)
        for tr, v, te_x, te_y in data_zip:
            for reg_idx, regularizer in enumerate(self.regularizers):
                for trial in range(self.ensemble_size):
                    #defined lstm model
                    lstm = Model(horizon, self.units, self.lstm, self.tanh, regularizer[0], regularizer[1], regularizer[2],
                                       padding)
                    #defined gru model
                    gru = Model(horizon, self.units, self.gru, self.tanh, regularizer[0], regularizer[1], regularizer[2],
                                      padding)

                    #compiled both models
                    lstm.compile(self.adam, self.mse_loss, self.metrics)
                    gru.compile(self.adam, self.mse_loss, self.metrics)

                    #fit both models and log the process
                    self.logger.info(f"Training LSTM fold {fold_idx} regularizer {reg_idx} ensemble {trial}")
                    lstm_hist = lstm.fit([tr, enc_names], [v[0], v[1], v[2]], self.epochs, verbose=0)
                    self.logger.info(f"Training GRU fold {fold_idx} regularizer {reg_idx} ensemble {trial}")
                    gru_hist = gru.fit([tr, enc_names], [v[0], v[1], v[2]], self.epochs, verbose=0)

                    # make predictions on validation data
                    lstm_pred_val = lstm.predict([tr, enc_names])
                    gru_pred_val = gru.predict([tr, enc_names])

                    # make predictions on test data
                    lstm_pred_test = lstm.predict([te_x, enc_names])
                    gru_pred_test = gru.predict([te_x, enc_names])

                    # rescale validation predictions
                    lstm_pred_val = self.prepare_predictions(lstm_pred_val, self.val_scalers[fold_idx])
                    gru_pred_val = self.prepare_predictions(gru_pred_val, self.val_scalers[fold_idx])

                    # rescale test predictions
                    lstm_pred_test = self.prepare_predictions(lstm_pred_test, self.test_scalers[fold_idx])
                    gru_pred_test = self.prepare_predictions(gru_pred_test, self.test_scalers[fold_idx])

                    # calculate RMSE for validation
                    lstm_rmse_val = self.calculate_rmse(orig_val[fold_idx], lstm_pred_val)
                    gru_rmse_val = self.calculate_rmse(orig_val[fold_idx], gru_pred_val)

                    # calculate RMSE for test
                    lstm_rmse_test = self.calculate_rmse(orig_test_y[fold_idx], lstm_pred_test)
                    gru_rmse_test = self.calculate_rmse(orig_test_y[fold_idx], gru_pred_test)

                    # calculate MAE for validation
                    lstm_mae_val = self.calculate_mae(orig_val[fold_idx], lstm_pred_val)
                    gru_mae_val = self.calculate_mae(orig_val[fold_idx], gru_pred_val)

                    # calculate MAE for test
                    lstm_mae_test = self.calculate_mae(orig_test_y[fold_idx], lstm_pred_test)
                    gru_mae_test = self.calculate_mae(orig_test_y[fold_idx], gru_pred_test)

                    lstm_name = f"fold_{fold_idx}_reg_{reg_idx}_ens_{trial}_lstm"
                    gru_name = f"fold_{fold_idx}_reg_{reg_idx}_ens_{trial}_gru"

                    # save model's predictions on validation data
                    self.save_to_npz(lstm_pred_val, lstm_name, "val_pred")
                    self.save_to_npz(gru_pred_val, gru_name, "val_pred")

                    # save model's predictions on test data
                    self.save_to_npz(lstm_pred_test, lstm_name, "test_pred")
                    self.save_to_npz(gru_pred_test, gru_name, "test_pred")

                    # save model's error scores for validation
                    self.save_to_npz(lstm_rmse_val, lstm_name, "rmse_val")
                    self.save_to_npz(gru_rmse_val, gru_name, "rmse_val")

                    self.save_to_npz(lstm_mae_val, lstm_name, "mae_val")
                    self.save_to_npz(gru_mae_val, gru_name, "mae_val")

                    # save model's error scores for test
                    self.save_to_npz(lstm_rmse_test, lstm_name, "rmse_test")
                    self.save_to_npz(gru_rmse_test, gru_name, "rmse_test")

                    self.save_to_npz(lstm_mae_test, lstm_name, "mae_test")
                    self.save_to_npz(gru_mae_test, gru_name, "mae_test")
            fold_idx += 1
            if fold_idx == 1:
              break

## Run Experiments

In [ ]:
# log results 
logging.basicConfig(filename="logger.log", format="%(asctime)s %(name)s %(levelname)s: %(message)s", level=logging.DEBUG)

console = logging.StreamHandler()
console.setLevel(logging.INFO)
console.setFormatter(logging.Formatter("%(asctime)s %(name)s %(levelname)s: %(message)s"))
logging.getLogger().addHandler(console)

dat = Data()
dat.prepare_data()

logging.info(f"Number of days in data: {len(dat.countries[0].data)}")
logging.info(f"Current date and time: {datetime.now()}")

exp = Experiment(dat.val_scalers, dat.test_y_scalers)
# run experiments defined in class above
exp.run_experiments(dat.horizon, dat.pad_val, dat.padded_scaled_train, dat.multi_out_scaled_val, dat.padded_scaled_test_x,
                    dat.multi_out_scaled_test_y, dat.val, dat.test_y, dat.encoded_names)

6/6 [==============================] - 1s 84ms/step


## Process Results

In [ ]:
TARGET_DIR = "cross_val_results"

# process the result files saved
def save_country_names
    # save country names
    d = Data()
    names = [country.name for country in d.countries]
    path = os.path.join(TARGET_DIR, "country_names.csv")
    with open(path, "w", newline="") as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(names) 

def load_country_names():
    #load country names
    path = os.path.join(TARGET_DIR, "country_names.csv")
    with open(path, "r", newline="") as csv_file:
        reader = csv.reader(csv_file)
        results = [row for row in reader]
    return results[0]

if __name__ == "__main__":

    save_country_names()

## Result Viewer

In [ ]:
# view results from saved files
#build file names according to make, variant of model
def build_file_name(fold, reg, ens, model, error, partition):
    return f"fold_{fold}_reg_{reg}_ens_{ens}_{model}_{error}_{partition}"
#load npz files
def load_npz(file_name):
    result_dir = "cross_val_results"

    file_path = os.path.join(result_dir, file_name)
    with np.load(file_path) as data:
        ret = [data[i] for i in data]  # There can be multiple arrays in a file.
    return np.squeeze(np.array(ret))
#format errors if any
def format_errors(fold, model, error, partition):
    ensemble_size = 10
    regularizers = 6
    results = []
    for regularizer in range(regularizers):
        mean_error = []
        for trial in range(ensemble_size):
            file_name = build_file_name(fold, regularizer, trial, model, error, partition)
            model_errors = load_npz(file_name)
            mean_error.append(np.mean(model_errors))
        results.append(mean_error)
    return np.stack(results).T
#draw box plots
def make_box_plot(lstm, gru, fold):
    def draw_plot(data, offset,edge_color, fill_color):
        pos = np.arange(data.shape[1])+offset
        bp = ax.boxplot(data, positions= pos, widths=0.3, showmeans=True, meanline=True, patch_artist=True, manage_ticks=False)
        for element in ['boxes', 'whiskers', 'fliers', 'medians', 'caps', 'means']:
            plt.setp(bp[element], color=edge_color)
        for patch in bp['boxes']:
            patch.set(facecolor=fill_color)
        return bp

    fig, ax = plt.subplots()
    bpA = draw_plot(lstm, -0.2, "black", "white")
    bpB = draw_plot(gru, +0.2,"grey", "white")
    plt.xticks(range(6))

    ax.set_xticklabels(["No reg", "L1", "L2", "Dropout", "ElasticNet", "All regs"])
    ax.set_ylabel("RMSE")
    ax.set_title(f"Comparison of models for validation fold {fold}")
    ax.legend([bpA["boxes"][0], bpB["boxes"][0]], ["LSTM", "GRU"])

    fig_path = os.path.join("plots", f"boxplot_{fold}")
    plt.savefig(fig_path)

def make_comparison_table_old(gru, lstm, fold):
    #make comparison table
    header = ["Model", "Type", "No reg", "L1", "L2", "Dropout", "ElasticNet", "All regs"]
    with open("model_comparison.csv", "a", newline="") as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(header)
        gru_mean_row = ["GRU", "Mean", *list(np.mean(gru, axis=0))]
        gru_var_row = ["GRU", "Var", *list(np.var(gru, axis=0))]
        lstm_mean_row = ["LSTM", "Mean", *list(np.mean(lstm, axis=0))]
        lstm_var_row = ["LSTM", "Var", *list(np.var(lstm, axis=0))]
        writer.writerow(gru_mean_row)
        writer.writerow(gru_var_row)
        writer.writerow(lstm_mean_row)
        writer.writerow(lstm_var_row)

def make_box_plots():
    #call the box plot function for a model
    folds = 14
    for fold in range(folds):
        gru_errors = format_errors(fold, "gru", "rmse", "test")
        lstm_errors = format_errors(fold, "lstm", "rmse", "test")
        make_box_plot(lstm_errors, gru_errors, fold)
        make_comparison_table_old(lstm_errors, gru_errors, fold)

def load_location_names():
    #load location names from results
    path = os.path.join("cross_val_results", "country_names.csv")
    with open(path, "r", newline="") as csv_file:
        reader = csv.reader(csv_file)
        results = [row for row in reader]
    return results[0]

def make_ensemble_plots(fold, reg, model, partition):
    #ensemble the plots
    preds = []
    errs = []
    ensemble_size = 10
    loc_names = load_location_names()
    display = (0, 10, 11)
    sub_titles = ["Confirmed", "Deceased", "Recovered"]
    orig = load_npz("test_y")

    for e in range(ensemble_size):
        pred_file_name = build_file_name(fold, reg, e, model, partition, "pred")
        error_file_name = build_file_name(fold, reg, e, model, "rmse", partition)
        preds.append(load_npz(pred_file_name))
        errs.append(load_npz(error_file_name))
    avg_pred = np.mean(preds, axis=0)
    avg_errs = np.mean(errs, axis=0)

    fig, axes = plt.subplots(1,3, figsize=(8,6))
    for loc_idx, loc_name in enumerate(loc_names):
        for idx, axis in enumerate(axes):
            axis.cla()
            for p in preds:
                axis.plot(p[loc_idx].T[idx], color="lightgrey", label="Ensemble")
            axis.plot(orig[fold][loc_idx].T[idx], color="black", label="Original data")
            axis.plot(avg_pred[loc_idx].T[idx], linestyle="--", color="black", label=f"Average {model.upper()}")
            axis.set_title(f"{sub_titles[idx]} : {round(avg_errs[loc_idx][idx], 3)} RMSE", pad=12)
        handles, labels = axes[2].get_legend_handles_labels()
        axes[0].set_ylabel("People")
        axes[1].set_xlabel("Days")
        axes[2].legend([handle for i, handle in enumerate(handles) if i in display],
                       [label for i, label in enumerate(labels) if i in display], loc=0)
        fig.suptitle(loc_name)
        plt.tight_layout()
        fig_path = os.path.join("plots", f"fold_{fold}_reg_{reg}_{model}_{partition}_{loc_name}")
        plt.savefig(fig_path, bbox_inches="tight")

def make_comparison_table(error_metric):
    # call comparison table function
    folds = 14
    regs = 6
    ens = 10
    err_table = []
    for fold in range(folds):
        gru_err_row = []
        lstm_err_row = []
        for reg in range(regs):
            gru_errs = []
            lstm_errs = []
            for ens_idx in range(ens):
                gru_f_name = build_file_name(fold, reg, ens_idx, "gru", error_metric, "test")
                lstm_f_name = build_file_name(fold, reg, ens_idx, "lstm", error_metric, "test")
                gru_errs_ens = load_npz(gru_f_name)
                lstm_errs_ens = load_npz(lstm_f_name)
                gru_errs.append(gru_errs_ens)
                lstm_errs.append(lstm_errs_ens)
            avg_gru_errs = np.mean(gru_errs)
            avg_lstm_errs = np.mean(lstm_errs)
            gru_err_row.append(avg_gru_errs)
            lstm_err_row.append(avg_lstm_errs)
        gru_err_row.append("GRU")
        lstm_err_row.append("LSTM")
        err_table.append(gru_err_row)
        err_table.append(lstm_err_row)
    print(tabulate(err_table, tablefmt='latex'))

#call the comparison table
make_comparison_table("mae")

\begin{tabular}{rrrrrrl}
\hline
  319.5   &  446.625 &  363.091 &  333.801 &  427.622 &  471.972 & GRU  \\
  359.726 &  401.704 &  340.661 &  311.689 &  421.056 &  595.859 & LSTM \\
  280.821 &  272.085 &  263.255 &  332.316 &  266.776 &  287.116 & GRU  \\
  287.663 &  284.511 &  292.158 &  288.097 &  276.452 &  351.49  & LSTM \\
  345.437 &  306.365 &  304.89  &  327.988 &  309.39  &  315.74  & GRU  \\
  338.742 &  354.014 &  348.156 &  331.267 &  406.766 &  344.074 & LSTM \\
  500.506 &  500.684 &  498.038 &  505.672 &  507.41  &  516.519 & GRU  \\
  518.53  &  523.206 &  540.011 &  499.124 &  523.93  &  524.885 & LSTM \\
  595.099 &  559.33  &  583.883 &  578.939 &  564.906 &  567.642 & GRU  \\
  589.371 &  557.974 &  593.289 &  585.837 &  569.296 &  577.883 & LSTM \\
  540.783 &  554.767 &  554.285 &  548.472 &  561.092 &  531.013 & GRU  \\
  544.32  &  538.935 &  540.229 &  545.547 &  560.693 &  526.886 & LSTM \\
  556.193 &  581.371 &  574.816 &  567.757 &  581.383 &  566.127 & G